In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sub = pd.read_csv('Sample_Submission.csv')

In [3]:
train.head(5)

,Offered_By,Category,Rating,Reviews,Size,Price,Content_Rating,Last_Updated_On,Release_Version,OS_Version_Required,Downloads
0,ps_id-24654,Finance,4.18,1481,Varies with device,Free,Everyone,May 05 2020,Varies with device,Varies with device,"100,000+"
1,ps_id-35329,Music And Audio,4.81,302,10M,Free,Everyone,Mar 26 2020,3.9.18,4.1 and up,"5,000+"
2,ps_id-11044,Game Casual,4.27,374,27M,Free,Everyone,May 01 2020,1.10.1,4.1 and up,"10,000+"
3,ps_id-36068,Business,4.03,122058,Varies with device,Free,Teen,May 02 2020,Varies with device,Varies with device,"10,000,000+"
4,ps_id-35831,Medical,4.60,358,Varies with device,297.5742,Everyone,Nov 29 2018,Varies with device,Varies with device,"5,000+"


In [4]:
test.tail(5)

,Offered_By,Category,Rating,Reviews,Size,Price,Content_Rating,Last_Updated_On,Release_Version,OS_Version_Required
24771,ps_id-30176,Shopping,3.74,38,21M,Free,Everyone,Feb 16 2020,2.4.20,4.1 and up
24772,ps_id-5357,Auto And Vehicles,4.80,4,4.5M,Free,Everyone,Jan 02 2019,1.3,5.0 and up
24773,ps_id-1920,Education,4.78,499,1.1M,Free,Everyone,Feb 16 2018,1.1,4.0.3 and up
24774,ps_id-6655,Game Puzzle,4.61,328050,72M,Free,Everyone,May 02 2020,2.5.3,4.1 and up
24775,ps_id-3916,Finance,3.98,1467,18M,Free,Everyone,Apr 30 2020,2.25.2,4.4 and up


In [5]:
sub.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0


In [6]:
train.isnull().sum(),test.isnull().sum(),train.shape,test.shape,train.dtypes

(Offered_By             0
 Category               0
 Rating                 0
 Reviews                0
 Size                   0
 Price                  0
 Content_Rating         0
 Last_Updated_On        0
 Release_Version        0
 OS_Version_Required    0
 Downloads              0
 dtype: int64,
 Offered_By             0
 Category               0
 Rating                 0
 Reviews                0
 Size                   0
 Price                  0
 Content_Rating         0
 Last_Updated_On        0
 Release_Version        0
 OS_Version_Required    0
 dtype: int64,
 (16516, 11),
 (24776, 10),
 Offered_By              object
 Category                object
 Rating                 float64
 Reviews                  int64
 Size                    object
 Price                   object
 Content_Rating          object
 Last_Updated_On         object
 Release_Version         object
 OS_Version_Required     object
 Downloads               object
 dtype: object)

In [7]:
df=train.append(test,ignore_index=True)

In [8]:
df['Size_type'] = df['Size'].str.strip().str[-1]
df['Size_type']

0        e
1        M
2        M
3        e
4        e
        ..
41287    M
41288    M
41289    M
41290    M
41291    M
Name: Size_type, Length: 41292, dtype: object

In [9]:
df['Size_type'] = df['Size'].str.strip().str[-1]
x=[]
for i in df['Size_type']:
  if i=='e':
    x.append(0)
  elif i=='M':
    x.append(1024)
  elif i=='k':
    x.append(1)
df['Size_type']=x
df['size_t']=df.Size.str.extract('(\d+)')
df['size_t']=df['size_t'].fillna(0)
df['Size']=df['Size_type'].astype(int)*df['size_t'].astype(int)
del df['Size_type']
del df['size_t']
df['Size']=df['Size'].replace(0,df['Size'].mean())

In [10]:
df['Size']

0        20254.44413
1        10240.00000
2        27648.00000
3        20254.44413
4        20254.44413
            ...     
41287    21504.00000
41288     4096.00000
41289     1024.00000
41290    73728.00000
41291    18432.00000
Name: Size, Length: 41292, dtype: float64

In [11]:
df['Price']=df['Price'].replace('Free',0).astype(float)

In [12]:
df['Last_Updated_On'] = pd.to_datetime(df['Last_Updated_On'])

In [13]:
k=['Last_Updated_On']
for i in k:
  df[i+'_'+'year'] = df[i].dt.year
  df[i+'_'+'day'] = df[i].dt.day
  df[i+'_'+'weekofyear'] = df[i].dt.weekofyear
  df[i+'_'+'month'] = df[i].dt.month
  df[i+'_'+'dayofweek'] =df[i].dt.dayofweek
  df[i+'_'+'weekend'] = (df[i].dt.weekday >=5).astype(int)
del df['Last_Updated_On']

C:\Users\kesha\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


In [14]:
df['Release_Version']=df['Release_Version'].replace('Varies with device','0')

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Release_Version'] = le.fit_transform(df['Release_Version'])
le.classes_

array(['0', '0.0.0-34-animals', '0.0.0-34-leaders', ..., 'Zs 5.2',
       '_18.08.01_ps', '_19.02.01-us_sy_ps'], dtype=object)

In [16]:
#df = pd.get_dummies(df, columns=['Category','Content_Rating','OS_Version_Required'])
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Category'] = le.fit_transform(df['Category'])
le.classes_
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Content_Rating'] = le.fit_transform(df['Content_Rating'])
le.classes_
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['OS_Version_Required'] = le.fit_transform(df['OS_Version_Required'])
le.classes_

array(['1.0 and up', '1.1 and up', '1.5 and up', '1.6 and up',
       '2.0 and up', '2.0.1 and up', '2.1 and up', '2.2 - 6.0',
       '2.2 and up', '2.3 - 3.2', '2.3 and up', '2.3.3 and up',
       '3.0 and up', '3.1 and up', '3.2 and up', '4.0 - 5.1',
       '4.0 and up', '4.0.3 - 7.1.1', '4.0.3 and up', '4.1 - 7.1.1',
       '4.1 - 8.0', '4.1 and up', '4.2 - 8.0', '4.2 and up', '4.3 and up',
       '4.4 and up', '4.4w and up', '5.0 - 6.0', '5.0 - 8.0',
       '5.0 and up', '5.1 and up', '6.0 and up', '7.0 - 7.1.1',
       '7.0 and up', '7.1 and up', '8.0 and up', 'Varies with device'],
      dtype=object)

In [17]:
df['Offered_By']=df.Offered_By.str.extract('(\d+)').astype(int)

In [18]:
train = df[df['Downloads'].isnull()==False]
test = df[df['Downloads'].isnull()==True]
del test['Downloads']

In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Downloads'] = le.fit_transform(train['Downloads'])
le.classes_

C:\Users\kesha\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


array(['1,000+', '1,000,000+', '1,000,000,000+', '10+', '10,000+',
       '10,000,000+', '100+', '100,000+', '100,000,000+', '5,000+',
       '5,000,000+', '5,000,000,000+', '50+', '50,000+', '50,000,000+',
       '500+', '500,000+', '500,000,000+'], dtype=object)

In [20]:
is_hol = train['Downloads'] == 11
df_try = train[is_hol]
train=train.append([df_try],ignore_index=True)

In [23]:
train_df=train
test_df=test

In [24]:
X = train_df.drop(labels=['Downloads'], axis=1)
y = train_df['Downloads'].values

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.10, random_state=101, stratify=y)

In [26]:
X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

((14865, 15), (14865,), (1652, 15), (1652,))

In [27]:
from sklearn.metrics import log_loss

In [28]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_cv, label=y_cv)

param = {'objective': 'multiclass',
         'num_class': 18,
         'boosting': 'dart',  
         'metric': 'multi_logloss',
         'learning_rate': 0.09, 
         'num_iterations': 400,
         'num_leaves': 31,
         'max_depth': -1,
         'min_data_in_leaf': 30,
         'bagging_fraction':0.9,
         'bagging_freq': 2,
         'feature_fraction': 0.9,
         'lambda_l2': 0.9,
         'min_data_per_group': 500,
         'max_bin': 255,
         'is_unbalance':True
         }

clf = lgb.train(params=param, 
                early_stopping_rounds=200,
                verbose_eval=100,
                train_set=train_data,
                valid_sets=[test_data])

y_pred = clf.predict(X_cv)

C:\Users\kesha\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\kesha\Anaconda3\lib\site-packages\lightgbm\callback.py:188: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


[100]	valid_0's multi_logloss: 1.19763
[200]	valid_0's multi_logloss: 1.1177
[300]	valid_0's multi_logloss: 1.08456
[400]	valid_0's multi_logloss: 1.07858


In [29]:
Xtest = test_df

In [34]:
y_pred = clf.predict(Xtest)

In [39]:
submission = pd.DataFrame(data=y_pred, columns=sub.columns)
submission = submission.round(decimals=2)
submission.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
24771,0.37,0.00,0.0,0.00,0.37,0.00,0.00,0.01,0.0,0.22,0.0,0.0,0.00,0.02,0.0,0.00,0.00,0.0
24772,0.36,0.01,0.0,0.06,0.01,0.00,0.25,0.00,0.0,0.15,0.0,0.0,0.01,0.00,0.0,0.14,0.00,0.0
24773,0.01,0.00,0.0,0.00,0.42,0.00,0.00,0.32,0.0,0.03,0.0,0.0,0.00,0.21,0.0,0.00,0.01,0.0
24774,0.00,0.01,0.0,0.00,0.00,0.88,0.00,0.00,0.0,0.00,0.1,0.0,0.00,0.00,0.0,0.00,0.00,0.0
24775,0.00,0.02,0.0,0.00,0.02,0.00,0.00,0.71,0.0,0.00,0.0,0.0,0.00,0.08,0.0,0.00,0.17,0.0


In [40]:
submission.to_csv('WH16.csv', index=False)

In [ ]:
from google.colab import files
files.download('WH16.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>